<a href="https://colab.research.google.com/github/nicovakr/Pytorch-init/blob/master/LinearReg_pytorch_builtsins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch.nn as nn
import torch 
import torchvision
import torchaudio
import numpy as np

In [8]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

In [9]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

In [10]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

## Dataset and DataLoader

In [11]:
from torch.utils.data import TensorDataset

In [23]:
#Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:5]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.],
         [102.,  43.,  37.],
         [ 69.,  96.,  70.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.],
         [ 22.,  37.],
         [103., 119.]]))

In [24]:
from torch.utils.data import DataLoader
#Data Loader will split the data into batches of predefined size

In [32]:
#Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle = True)

In [33]:
for xb, yb in train_dl:
  print(xb)
  print(yb)
  break

tensor([[ 87., 135.,  57.],
        [ 69.,  96.,  70.],
        [ 73.,  66.,  44.],
        [ 73.,  67.,  43.],
        [ 91.,  88.,  64.]])
tensor([[118., 134.],
        [103., 119.],
        [ 57.,  69.],
        [ 56.,  70.],
        [ 81., 101.]])


## nn.Linear

In [34]:
#Define Model
model = nn.Linear(3,2) #(in_features, out_features)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.4157, -0.5651, -0.3230],
        [ 0.3751, -0.2286,  0.4299]], requires_grad=True)
Parameter containing:
tensor([ 0.3334, -0.5064], requires_grad=True)


In [35]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.4157, -0.5651, -0.3230],
         [ 0.3751, -0.2286,  0.4299]], requires_grad=True),
 Parameter containing:
 tensor([ 0.3334, -0.5064], requires_grad=True)]

In [36]:
#Generate predictions
preds = model(inputs)
preds

tensor([[ -81.7583,   30.0393],
        [-107.8892,   41.0166],
        [-130.2817,   26.4192],
        [ -78.3139,   43.8242],
        [-105.2024,   33.5154],
        [ -81.6089,   30.6430],
        [-107.6471,   41.6751],
        [-131.0204,   27.2242],
        [ -78.4633,   43.2205],
        [-105.1097,   33.5703],
        [ -81.5162,   30.6978],
        [-107.7398,   41.6203],
        [-130.5238,   25.7607],
        [ -78.4067,   43.7693],
        [-105.3518,   32.9117]], grad_fn=<AddmmBackward0>)

## Loss Function

In [37]:
import torch.nn.functional as F

In [38]:
#the nn.functional package contains many useful loss functions 

#define loss function
loss_fn = F.mse_loss

In [39]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(19375.1855, grad_fn=<MseLossBackward0>)


## Optimizer

In [40]:
#optimizer optim.SGD : Stochastic because samples are selected in 
#                      random batches instead of as a single group

# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

## Train the model

In [41]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):

  # Repeat for given number of epochs
  for epoch in range(num_epochs):

    # Train with batches of data
    for xb, yb in train_dl:

      # 1. Generate predictions
      pred = model(xb)

      # 2. Calculate Loss
      loss = loss_fn(pred, yb)

      # 3. Compute Gradients
      loss.backward()

      # 4. Update parameters using gradients
      opt.step()

      # 5. Reset the gradients to zero
      opt.zero_grad()

    # Print the progress
    if (epoch+1) % 10 == 0:
      print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))


# instead of updating parameters Weight and Bias manually we use opt.step to do



In [48]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 5.7585
Epoch [20/100], Loss: 4.2542
Epoch [30/100], Loss: 4.8285
Epoch [40/100], Loss: 3.1524
Epoch [50/100], Loss: 4.2810
Epoch [60/100], Loss: 2.9926
Epoch [70/100], Loss: 2.3256
Epoch [80/100], Loss: 5.0771
Epoch [90/100], Loss: 4.2891
Epoch [100/100], Loss: 3.6668


In [49]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 57.1601,  70.3763],
        [ 80.8666, 100.0585],
        [120.5186, 133.7886],
        [ 21.5659,  38.1936],
        [ 99.4621, 117.3183],
        [ 55.8900,  69.2956],
        [ 80.5101, 100.0625],
        [120.6939, 134.3469],
        [ 22.8360,  39.2743],
        [100.3757, 118.4031],
        [ 56.8036,  70.3803],
        [ 79.5965,  98.9778],
        [120.8751, 133.7846],
        [ 20.6523,  37.1088],
        [100.7322, 118.3991]], grad_fn=<AddmmBackward0>)

In [44]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

In [45]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.2340,  0.8608,  0.4074],
         [-0.1217,  0.7319,  0.7506]], requires_grad=True),
 Parameter containing:
 tensor([ 0.3394, -0.5085], requires_grad=True)]